주성분 분석이란 고차원 데이터를 저차원 데이터로 축소하는 알고리즘을 말한다.
주로 고차원의 데이터를 3차원 이하의 데이터로 바꿔 시각화하는데 많이 사용되며, 유용한 정보만 살려서 적은 메모리에 저장하거나 데이터의 노이즈를 줄이고 싶을 때도 사용되는 알고리즘이다.

주성분 분석의 특징은 데이터의 분산을 최대한 유지하면서 저차원으로 데이터를 변환하는데 있다. 분산을 유지하는 이유는 데이터의 고유한 특성을 최대한 유지시키기 위해서이다.

In [5]:
import warnings
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 평균을 0으로, 분산을 1로 조정해서 표준화를 하기 위해 StandardScaler를 import한다.
from sklearn.preprocessing import StandardScaler

식습관 데이터를 차원축소시켜 시각화하기

In [3]:
df = pd.DataFrame(columns = ['calorie', 'breakfast', 'lunch', 'dinner', 'exercise', 'body_shape'])
df.loc[0] = [1200, 1, 0, 0, 2, 'Skinny']
df.loc[1] = [2800, 1, 1, 1, 1, 'Normal']
df.loc[2] = [3500, 2, 2, 1, 0, 'Fat']
df.loc[3] = [1400, 0, 1, 0, 3, 'Skinny']
df.loc[4] = [5000, 2, 2, 2, 0, 'Fat']
df.loc[5] = [1300, 0, 0, 1, 2, 'Skinny']
df.loc[6] = [3000, 1, 0, 1, 1, 'Normal']
df.loc[7] = [4000, 2, 2, 2, 0, 'Fat']
df.loc[8] = [2600, 0, 2, 0, 0, 'Normal']
df.loc[9] = [3000, 1, 2, 1, 1, 'Fat']

데이터 전처리
데이터 전처리 과정을 통해 데이터 범위가 제각각인 특징을 표준화한다.
데이터에 데이터의 특징(피쳐)과 레이블(클래스, 타겟)가 함께있으므로 데이터의 특징으로만 구성된 X 데이터프레임을 만든다.

In [4]:
X = df[['calorie', 'breakfast', 'lunch', 'dinner', 'exercise']]
X

calorie breakfast lunch dinner exercise
0    1200         1     0      0        2
1    2800         1     1      1        1
2    3500         2     2      1        0
3    1400         0     1      0        3
4    5000         2     2      2        0
5    1300         0     0      1        2
6    3000         1     0      1        1
7    4000         2     2      2        0
8    2600         0     2      0        0
9    3000         1     2      1        1

표준화(정규화)
위의 데이터 전처리 결과를 통해 칼로리가 유난히 다른 특성들에 비해 수치 범위가 큰 것을 확인할 수 있다.
수치가 큰 칼로리와 다른 특성들은 우리가 눈으로 비교할 수 없듯이 머신러닝 모델도 쉽게 비교하기 힘들어 한다.
이 경우 모든 특성들이 평균이 0이고 표준편차가 1인 데이터로 바군 후 비교하면 특성들의 상관관계를 이해하기 쉬워지고 머신러닝 모델 역시 더욱 효율적으로 데이터를 학습할 수 있다.

In [6]:
# 칼로리를 포함한 모든 특성을 같은 범위내로 표준화한다.
x_std = StandardScaler().fit_transform(X)
print(type(x_std))
print(x_std)

<class 'numpy.ndarray'>
[[-1.35205803  0.         -1.3764944  -1.28571429  1.        ]
 [ 0.01711466  0.         -0.22941573  0.14285714  0.        ]
 [ 0.61612771  1.29099445  0.91766294  0.14285714 -1.        ]
 [-1.18091145 -1.29099445 -0.22941573 -1.28571429  2.        ]
 [ 1.89972711  1.29099445  0.91766294  1.57142857 -1.        ]
 [-1.26648474 -1.29099445 -1.3764944   0.14285714  1.        ]
 [ 0.18826125  0.         -1.3764944   0.14285714  0.        ]
 [ 1.04399418  1.29099445  0.91766294  1.57142857 -1.        ]
 [-0.15403193 -1.29099445  0.91766294 -1.28571429 -1.        ]
 [ 0.18826125  0.          0.91766294  0.14285714  0.        ]]


레이블 분리하기
각 데이터 특성에 따른 레이블(체형)을 따로 Y 데이터프레임으로 생성한다.

In [7]:
Y = df['body_shape']
Y

0    Skinny
1    Normal
2       Fat
3    Skinny
4       Fat
5    Skinny
6    Normal
7       Fat
8    Normal
9       Fat
Name: body_shape, dtype: object

공분산 행렬 구하기
주성분 분석을 하기 위해 가장 먼저 할 일은 특징들의 공분산 행렬을 구하는 것이다.

In [8]:
# np.cov() 함수를 사용하면 공분산을 계산할 수 있다.
features = x_std.T # 전치
covariance_matrix = np.cov(features)

고유값과 고유벡터 구하기
공분산 행렬에서 가장 큰 고유값을 가진 고유벡터를 찾아 그 벡터 위에 데이터를 사영시킨다.
사영시킨다는 의미는 데이터 손실을 최소한으로 하는 2차원을 1차원 공간으로 축소한다는 의미이다.

In [9]:
# 공분산 행렬에서 고유값과 고유벡터를 계산한다.
eig_vals, eig_vecs = np.linalg.eig(covariance_matrix)
print('고유값\n', eig_vals, sep='')
print('고유벡터\n', eig_vecs, sep='')

고유값
[4.0657343  0.8387565  0.07629538 0.27758568 0.2971837 ]
고유벡터
[[-0.508005   -0.0169937  -0.84711404  0.11637853  0.10244985]
 [-0.44660335 -0.36890361  0.12808055 -0.63112016 -0.49973822]
 [-0.38377913  0.70804084  0.20681005 -0.40305226  0.38232213]
 [-0.42845209 -0.53194699  0.3694462   0.22228235  0.58954327]
 [ 0.46002038 -0.2816592  -0.29450345 -0.61341895  0.49601841]]


In [10]:
# 가장 큰 고유벡터로 데이터를 사영할 경우 얼마만큼의 정보가 유지되는지 확인한다.
for i in range(len(eig_vals)):
    print(eig_vals[i]/ sum(eig_vals))

0.7318321731427543
0.1509761705671511
0.013733167936950091
0.04996542317586189
0.053493065177282625


결과를 통해 1차원 데이터로 차원 축소를 하더라고 73% 정도에 해당되는 정보를 유지할 수 있다는 사실을 확인할 수 있다. 27% 정도의 정보 손실은 감수하고 1차원 데이터로 축소한다.

In [13]:
projected_X = x_std.dot(eig_vecs.T[0]) / np.linalg.norm(eig_vecs.T[0])
projected_X

array([ 2.22600943,  0.0181432 , -1.76296611,  2.73542407, -3.02711544,
        2.14702579,  0.37142473, -2.59239883,  0.39347815, -0.50902498])

In [14]:
import seaborn as sns


result['label'] = Y
result